In [ ]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import boto3
import pickle

In [ ]:
df_track = pd.read_csv('features_wGenre.csv', index_col=0)

In [ ]:
print(df_track.shape)
df_track.head()

In [ ]:
# df_track.columns

In [ ]:
to_drop = ['id', 'uri', 'track_href', 'analysis_url', 'type']
df_track = df_track.drop(to_drop, axis=1)

In [ ]:
print(df_track.shape)
df_track.head()

In [ ]:
df_track = df_track[(df_track['playlist id'] != '3 x 5') & (df_track['playlist id'] != 'Loss for words')]
df_track = df_track.reset_index(drop=True)

In [ ]:
profile = ProfileReport(df_track, title="Pandas Profiling Report")
# profile.to_notebook_iframe()

In [ ]:
c = 0.0000001
df_track[['speechiness', 'acousticness', 'instrumentalness']] = df_track[['speechiness', 'acousticness', 'instrumentalness']] + c
df_track[['speechiness', 'acousticness', 'instrumentalness']] = np.log(df_track[['speechiness', 'acousticness', 'instrumentalness']])

In [ ]:
df_track = pd.get_dummies(df_track, prefix=['key', 'time_signature', 'genre'], columns=['key', 'time_signature', 'genre'])

In [ ]:
# df_track.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_track.drop('playlist id', axis=1), df_track['playlist id'], test_size=0.2, random_state=2021)

In [ ]:
X_train.head()

In [ ]:
scale_col = ['danceability', 'energy', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms']

scaler = StandardScaler()
X_train[scale_col] = scaler.fit_transform(X_train[scale_col])

In [ ]:
pickle_byte_obj = pickle.dumps(scaler) 
s3_resource = boto3.resource('s3')
_ = s3_resource.Object('spotify-net', 'scaler').put(Body=pickle_byte_obj)

In [ ]:
X_train.head()

In [ ]:
profile = ProfileReport(X_train, title="Pandas Profiling Report")
# profile.to_notebook_iframe()

In [ ]:
# What about correlations between genres? Cast into lower dimension?

In [ ]:
df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

In [ ]:
df_train.to_csv('s3://spotify-net/features_train.csv')
df_test.to_csv('s3://spotify-net/features_test.csv')